In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [6]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [171]:
def generate(instruction, knowledge, dialog):
    knowledge = ' <|knowledge|> ' + knowledge[0] + " <|knowledge|> " + knowledge[1]
    dialog = ' '.join(dialog)
    query = f"<|instruction|> {instruction} <|context|> {dialog} {knowledge} => "
    input_ids = tokenizer(f"{query}", return_tensors="pt").input_ids
    print(f"Length of input_ids: {len(input_ids[0])}")
    outputs = model.generate(input_ids, max_new_tokens=64, min_length=2, num_beams=5, do_sample=False)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output

In [172]:
# Instruction for a chitchat task
instruction = f'Given possible responses and a dialogue context, complete the context as the system.'
# Provide knowldge
knowledge = [
    "User: Ok. Can I get the phone number? And could you recommend me a hotel? System: Sure, here you go: 01234567890. Regarding the hotel, I would recommend the Gran Prestige in the center of town. User: Seems good. Can you make a reservation? System: Sure, when would you like that reservation?",
    "User: Can you give me the phone number and recommend me a restaurant? System: As for the restaurant, I recommend you the Quality Food in the city center. User: Can you book it? System: Yes, when do you want that reservation?"
]

dialog = [
  "User: Sounds good, could I get that phone number? Also, could you recommend me an expensive hotel?",
  "System: Bedouin's phone is 01223367660. As far as hotels go, I recommend the University Arms Hotel in the center of town.",
  "User: Yes. Can you book it for me?",
]

In [173]:
!CUDA_VISIBLE_DEVICES=3
response = generate(instruction, knowledge, dialog)
print(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Length of input_ids: 245
System recommends the University Arms Hotel in the center of town, the Gran Prestige in the center of town, and the Quality Food in the city center.
